# Process Accessibility Time Series Data

In [1]:
# Import packages
import os
import glob
import shapely
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import mapclassify as mc
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap

Step 1: Load Shapefile Data and Get Date List

In [2]:
# Define file path
modelFolderPath = "Accessibility_Dissolve_Animation"

In [3]:
# Read updated shapefile and get the date list
chicagoNameList = glob.glob(os.path.join(modelFolderPath,"Chicago_ACC.??-??-20??-*.shp"))
illinoisNameList = glob.glob(os.path.join(modelFolderPath,"Illinois_ACC.??-??-20??-*.shp"))
chicagoList = []
illinoisList = []

for i in range(0, len(chicagoNameList)):
    chicagoList.append(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC.??-??-20??-*.shp"))[i][45:55])
for i in range(0, len(illinoisNameList)):
    illinoisList.append(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC.??-??-20??-*.shp"))[i][46:56])
    
print(chicagoList)
print(illinoisList)

['05-29-2020', '06-29-2020', '08-31-2020', '09-28-2020', '09-08-2020', '07-09-2020', '05-28-2020', '09-21-2020', '06-01-2020', '08-05-2020', '07-14-2020', '10-05-2020', '06-30-2020', '10-03-2020', '06-19-2020', '09-02-2020', '06-27-2020', '05-27-2020', '11-14-2020', '05-26-2020', '09-13-2020', '07-15-2020', '10-07-2020', '07-26-2020', '06-13-2020', '08-06-2020', '09-29-2020', '08-14-2020', '06-15-2020', '06-17-2020', '07-13-2020', '06-22-2020', '11-22-2020', '10-01-2020', '07-03-2020', '06-10-2020', '08-01-2020', '07-12-2020', '08-10-2020', '11-29-2020', '09-05-2020', '08-17-2020', '09-19-2020', '07-11-2020', '08-23-2020', '06-02-2020', '07-31-2020', '07-03-2020', '08-18-2020', '07-21-2020', '09-11-2020', '09-20-2020', '07-22-2020', '05-23-2020', '08-11-2020', '06-07-2020', '10-04-2020', '08-02-2020', '11-16-2020', '08-20-2020', '11-13-2020', '08-21-2020', '11-15-2020', '07-28-2020', '08-19-2020', '08-27-2020', '05-30-2020', '08-28-2020', '12-01-2020', '08-09-2020', '11-28-2020', '12-0

Step 2: Specify the Start and End of Time Series

In [4]:
# Define functions to select the first and last date
def selectFirstDate(nameList):
    dmin = datetime.strptime(nameList[0], "%m-%d-%Y").date()
    index = 0
    for i in range(0, len(nameList)):
        d = datetime.strptime(nameList[i], "%m-%d-%Y").date()
        if (d<=dmin):
            index = i
            dmin = d
    return index

def selectLastDate(nameList):
    dmax = datetime.strptime(nameList[0], "%m-%d-%Y").date()
    index = 0
    for i in range(0, len(nameList)):
        d = datetime.strptime(nameList[i], "%m-%d-%Y").date()
        if (d>=dmax):
            index = i
            dmax = d
    return index

Step 3: Classify Data Based on Natural Breaks and Dissolve the Outcomes by Category

In [5]:
# Define the classification bins and generate time series data for ICU beds accessibility in Chicago Area
firstIndex = selectFirstDate(chicagoList)
lastIndex = selectLastDate(chicagoList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[lastIndex]+"*.shp"))[0])
nbArray_i = mc.NaturalBreaks(df['hospital_i'], k=6).bins.astype(float)

def classifyChicago(value):
    if value >= 0 and value <= nbArray_i[0]:
        return 0
    if value >= nbArray_i[0] and value <= nbArray_i[1]:
        return 1
    elif value > nbArray_i[1] and value <= nbArray_i[2]:
        return 2
    elif value > nbArray_i[2] and value <= nbArray_i[3]:
        return 3
    elif value > nbArray_i[3] and value <= nbArray_i[4]:
        return 4
    else:
        return 5

def classRangeChicago(value):
    if value == 0:
        return "0~"+str(round(nbArray_i[0],2))
    if value == 1:
        return str(round(nbArray_i[0],2))+"~"+str(round(nbArray_i[1],2))
    elif value == 2:
        return str(round(nbArray_i[1],2))+"~"+str(round(nbArray_i[2],2))
    elif value == 3:
        return str(round(nbArray_i[2],2))+"~"+str(round(nbArray_i[3],2))
    elif value == 4:
        return str(round(nbArray_i[3],2))+"~"+str(round(nbArray_i[4],2))
    else:
        return str(round(nbArray_i[4],2))+"~"+str(round(nbArray_i[5],2))

chicagoACC_dissolve_list = []    

for i in range(0, len(chicagoList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (chicagoList[i] == chicagoList[i-1]):
            continue
    chicagoACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[i]+"*.shp"))[0])
    
    chicagoACC['category'] = chicagoACC.apply(lambda x: classifyChicago(x.hospital_i), axis = 1)
    chicagoACC_dissolve = chicagoACC.dissolve(by='category', aggfunc = 'mean')
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['id'])
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['hospital_v'])
    chicagoACC_dissolve["category"] = range(0, len(chicagoACC_dissolve))
    chicagoACC_dissolve["start"] = (datetime.strptime(chicagoList[i], "%m-%d-%Y")-timedelta(days=1)).date().strftime("%Y-%m-%d")
    chicagoACC_dissolve["end"] = datetime.strptime(chicagoList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    
    chicagoACC_dissolve = chicagoACC_dissolve.to_crs({'init': 'epsg:4326'})
    #chicagoACC_dissolve.to_file("Chicago_ACC."+chicagoList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    chicagoACC_dissolve_list.append(chicagoACC_dissolve)
    
chicagoACC_concat = pd.concat(chicagoACC_dissolve_list, axis=0)
chicagoACC_concat = chicagoACC_concat.drop(columns=['category'])
chicagoACC_concat = chicagoACC_concat.reset_index()

#delta = timedelta(days=len(chicagoList))
firstDate = datetime.strptime(chicagoList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(chicagoList[lastIndex], "%m-%d-%Y").date()
chicagoACC_concat["dt_unit"] = "day"
chicagoACC_concat['geometry'] = chicagoACC_concat.apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x.geometry, rounding_precision=8)).simplify(0), axis = 1)
chicagoACC_concat.to_file("Chicago_ACC_i.geojson", driver="GeoJSON")

print("done")

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://

done


In [6]:
chicagoACC_concat

,category,geometry,hospital_i,start,end,dt_unit
0,0,"MULTIPOLYGON (((-87.53356 41.64289, -87.53184 ...",0.064171,2020-05-29,2020-05-29,day
1,1,"MULTIPOLYGON (((-87.53359 41.64739, -87.53184 ...",0.376355,2020-05-29,2020-05-29,day
2,2,"MULTIPOLYGON (((-87.60981 41.64251, -87.61152 ...",0.543679,2020-05-29,2020-05-29,day
3,3,"MULTIPOLYGON (((-87.70545 41.67124, -87.70369 ...",0.675560,2020-05-29,2020-05-29,day
4,4,"MULTIPOLYGON (((-87.64889 41.73687, -87.64718 ...",0.783666,2020-05-29,2020-05-29,day
...,...,...,...,...,...,...
1015,1,"MULTIPOLYGON (((-87.59595 41.64258, -87.59424 ...",0.374080,2020-09-25,2020-09-25,day
1016,2,"MULTIPOLYGON (((-87.60121 41.64931, -87.59946 ...",0.542953,2020-09-25,2020-09-25,day
1017,3,"MULTIPOLYGON (((-87.71409 41.66893, -87.71580 ...",0.676173,2020-09-25,2020-09-25,day
1018,4,"MULTIPOLYGON (((-87.64889 41.73687, -87.64718 ...",0.784027,2020-09-25,2020-09-25,day


In [7]:
# Define the classification bins and generate time series data for ventilators accessibility in Chicago Area
firstIndex = selectFirstDate(chicagoList)
lastIndex = selectLastDate(chicagoList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[lastIndex]+"*.shp"))[0])
nbArray_v = mc.NaturalBreaks(df['hospital_v'], k=6).bins.astype(float)

def classifyChicago(value):
    if value >= 0 and value <= nbArray_v[0]:
        return 0
    if value >= nbArray_v[0] and value <= nbArray_v[1]:
        return 1
    elif value > nbArray_v[1] and value <= nbArray_v[2]:
        return 2
    elif value > nbArray_v[2] and value <= nbArray_v[3]:
        return 3
    elif value > nbArray_v[3] and value <= nbArray_v[4]:
        return 4
    else:
        return 5

def classRangeChicago(value):
    if value == 0:
        return "0~"+str(round(nbArray_v[0],2))
    if value == 1:
        return str(round(nbArray_v[0],2))+"~"+str(round(nbArray_v[1],2))
    elif value == 2:
        return str(round(nbArray_v[1],2))+"~"+str(round(nbArray_v[2],2))
    elif value == 3:
        return str(round(nbArray_v[2],2))+"~"+str(round(nbArray_v[3],2))
    elif value == 4:
        return str(round(nbArray_v[3],2))+"~"+str(round(nbArray_v[4],2))
    else:
        return str(round(nbArray_v[4],2))+"~"+str(round(nbArray_v[5],2))

chicagoACC_dissolve_list = []    

for i in range(0, len(chicagoList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (chicagoList[i] == chicagoList[i-1]):
            continue
    chicagoACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[i]+"*.shp"))[0])
    #chicagoACC = chicagoACC.rename(columns={"normal_cov": "normal_val"})
    
    chicagoACC['category'] = chicagoACC.apply(lambda x: classifyChicago(x.hospital_v), axis = 1)
    chicagoACC_dissolve = chicagoACC.dissolve(by='category', aggfunc = 'mean')
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['id'])
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['hospital_i'])
    chicagoACC_dissolve["category"] = range(0, len(chicagoACC_dissolve))
    chicagoACC_dissolve["start"] = (datetime.strptime(chicagoList[i], "%m-%d-%Y")-timedelta(days=1)).date().strftime("%Y-%m-%d")
    chicagoACC_dissolve["end"] = datetime.strptime(chicagoList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    
    chicagoACC_dissolve = chicagoACC_dissolve.to_crs({'init': 'epsg:4326'})
    #chicagoACC_dissolve.to_file("Chicago_ACC."+chicagoList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    chicagoACC_dissolve_list.append(chicagoACC_dissolve)
    
chicagoACC_concat = pd.concat(chicagoACC_dissolve_list, axis=0)
chicagoACC_concat = chicagoACC_concat.drop(columns=['category'])
chicagoACC_concat = chicagoACC_concat.reset_index()

#delta = timedelta(days=len(chicagoList))
firstDate = datetime.strptime(chicagoList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(chicagoList[lastIndex], "%m-%d-%Y").date()
chicagoACC_concat["dt_unit"] = "day"
chicagoACC_concat['geometry'] = chicagoACC_concat.apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x.geometry, rounding_precision=8)).simplify(0), axis = 1)
chicagoACC_concat.to_file("Chicago_ACC_v.geojson", driver="GeoJSON")

print("done")

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://

done


In [8]:
chicagoACC_concat

,category,geometry,hospital_v,start,end,dt_unit
0,0,"MULTIPOLYGON (((-87.52838 41.64517, -87.52666 ...",0.058374,2020-05-29,2020-05-30,day
1,1,"MULTIPOLYGON (((-87.52838 41.64517, -87.52662 ...",0.374407,2020-05-29,2020-05-30,day
2,2,"MULTIPOLYGON (((-87.60981 41.64251, -87.61152 ...",0.541377,2020-05-29,2020-05-30,day
3,3,"MULTIPOLYGON (((-87.70545 41.67124, -87.70369 ...",0.671067,2020-05-29,2020-05-30,day
4,4,"MULTIPOLYGON (((-87.64898 41.74588, -87.64722 ...",0.779416,2020-05-29,2020-05-30,day
...,...,...,...,...,...,...
1015,1,"MULTIPOLYGON (((-87.53359 41.64739, -87.53184 ...",0.373500,2020-09-25,2020-09-26,day
1016,2,"MULTIPOLYGON (((-87.60981 41.64251, -87.61152 ...",0.541719,2020-09-25,2020-09-26,day
1017,3,"MULTIPOLYGON (((-87.71409 41.66893, -87.71580 ...",0.673397,2020-09-25,2020-09-26,day
1018,4,"MULTIPOLYGON (((-87.64889 41.73687, -87.64718 ...",0.780551,2020-09-25,2020-09-26,day


In [9]:
# Define the classification bins and generate time series data for ICU beds accessibility in Illinois Area
firstIndex = selectFirstDate(illinoisList)
lastIndex = selectLastDate(illinoisList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[lastIndex]+"*.shp"))[0])
nbArray_i = mc.NaturalBreaks(df['hospital_i'], k=6).bins.astype(float)

def classifyIllinois(value):   
    if value >= 0 and value <= nbArray_i[0]:
        return 0
    if value >= nbArray_i[0] and value <= nbArray_i[1]:
        return 1
    elif value > nbArray_i[1] and value <= nbArray_i[2]:
        return 2
    elif value > nbArray_i[2] and value <= nbArray_i[3]:
        return 3
    elif value > nbArray_i[3] and value <= nbArray_i[4]:
        return 4
    else:
        return 5
    
def classRangeIllinois(value):
    if value == 0:
        return "0~"+str(round(nbArray_i[0],2))
    if value == 1:
        return str(round(nbArray_i[0],2))+"~"+str(round(nbArray_i[1],2))
    elif value == 2:
        return str(round(nbArray_i[1],2))+"~"+str(round(nbArray_i[2],2))
    elif value == 3:
        return str(round(nbArray_i[2],2))+"~"+str(round(nbArray_i[3],2))
    elif value == 4:
        return str(round(nbArray_i[3],2))+"~"+str(round(nbArray_i[4],2))
    else:
        return str(round(nbArray_i[4],2))+"~"+str(round(nbArray_i[5],2))

illinoisACC_dissolve_list = []    

for i in range(0, len(illinoisList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (illinoisList[i] == illinoisList[i-1]):
            continue
    illinoisACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[i]+"*.shp"))[0])
    #illinoisACC = illinoisACC.rename(columns={"normal_cov": "normal_val"})
    
    illinoisACC['category'] = illinoisACC.apply(lambda x: classifyIllinois(x.hospital_i), axis = 1)
    illinoisACC_dissolve = illinoisACC.dissolve(by='category', aggfunc = 'mean')
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['id'])
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['hospital_v'])
    illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    illinoisACC_dissolve["start"] = (datetime.strptime(illinoisList[i], "%m-%d-%Y")-timedelta(days=1)).date().strftime("%Y-%m-%d")
    illinoisACC_dissolve["end"] = datetime.strptime(illinoisList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    
    illinoisACC_dissolve = illinoisACC_dissolve.to_crs({'init': 'epsg:4326'})
    #illinoisACC_dissolve.to_file("Illinois_ACC."+illinoisList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    illinoisACC_dissolve_list.append(illinoisACC_dissolve)
    
illinoisACC_concat = pd.concat(illinoisACC_dissolve_list, axis=0)
illinoisACC_concat = illinoisACC_concat.drop(columns=['category'])
illinoisACC_concat = illinoisACC_concat.reset_index()

#delta = timedelta(days=len(illinoisList))
firstDate = datetime.strptime(illinoisList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(illinoisList[lastIndex], "%m-%d-%Y").date()
illinoisACC_concat["dt_unit"] = "day"
illinoisACC_concat['geometry'] = illinoisACC_concat.apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x.geometry, rounding_precision=8)).simplify(0), axis = 1)
illinoisACC_concat.to_file("Illinois_ACC_i.geojson", driver="GeoJSON")

print("done")

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://

done


In [10]:
illinoisACC_concat

,category,geometry,hospital_i,start,end,dt_unit
0,0,"MULTIPOLYGON (((-88.43522 37.07011, -88.45102 ...",0.011013,2020-05-29,2020-05-30,day
1,1,"MULTIPOLYGON (((-89.17515 37.39681, -89.19079 ...",0.053105,2020-05-29,2020-05-30,day
2,2,"MULTIPOLYGON (((-90.35500 39.66639, -90.37070 ...",0.121597,2020-05-29,2020-05-30,day
3,3,"MULTIPOLYGON (((-88.01679 38.53902, -88.03303 ...",0.220248,2020-05-29,2020-05-30,day
4,4,"MULTIPOLYGON (((-88.33431 38.69353, -88.31813 ...",0.345745,2020-05-29,2020-05-30,day
...,...,...,...,...,...,...
978,1,"MULTIPOLYGON (((-88.01490 38.40386, -88.03111 ...",0.049107,2020-08-08,2020-08-09,day
979,2,"MULTIPOLYGON (((-87.96802 38.60702, -87.98429 ...",0.124384,2020-08-08,2020-08-09,day
980,3,"MULTIPOLYGON (((-88.52113 38.89400, -88.53728 ...",0.197350,2020-08-08,2020-08-09,day
981,4,"MULTIPOLYGON (((-87.98614 38.71952, -87.96984 ...",0.313643,2020-08-08,2020-08-09,day


In [11]:
# Define the classification bins and generate time series data for ventilators accessibility in Illinois Area
firstIndex = selectFirstDate(illinoisList)
lastIndex = selectLastDate(illinoisList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[lastIndex]+"*.shp"))[0])
nbArray_v = mc.NaturalBreaks(df['hospital_v'], k=6).bins.astype(float)

def classifyIllinois(value):   
    if value >= 0 and value <= nbArray_v[0]:
        return 0
    if value >= nbArray_v[0] and value <= nbArray_v[1]:
        return 1
    elif value > nbArray_v[1] and value <= nbArray_v[2]:
        return 2
    elif value > nbArray_v[2] and value <= nbArray_v[3]:
        return 3
    elif value > nbArray_v[3] and value <= nbArray_v[4]:
        return 4
    else:
        return 5
    
def classRangeIllinois(value):
    if value == 0:
        return "0~"+str(round(nbArray_v[0],2))
    if value == 1:
        return str(round(nbArray_v[0],2))+"~"+str(round(nbArray_v[1],2))
    elif value == 2:
        return str(round(nbArray_v[1],2))+"~"+str(round(nbArray_v[2],2))
    elif value == 3:
        return str(round(nbArray_v[2],2))+"~"+str(round(nbArray_v[3],2))
    elif value == 4:
        return str(round(nbArray_v[3],2))+"~"+str(round(nbArray_v[4],2))
    else:
        return str(round(nbArray_v[4],2))+"~"+str(round(nbArray_v[5],2))

illinoisACC_dissolve_list = []    

for i in range(0, len(illinoisList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (illinoisList[i] == illinoisList[i-1]):
            continue
    illinoisACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[i]+"*.shp"))[0])
    #illinoisACC = illinoisACC.rename(columns={"normal_cov": "normal_val"})
    
    illinoisACC['category'] = illinoisACC.apply(lambda x: classifyIllinois(x.hospital_v), axis = 1)
    illinoisACC_dissolve = illinoisACC.dissolve(by='category', aggfunc = 'mean')
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['id'])
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['hospital_i'])
    illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    illinoisACC_dissolve["start"] = (datetime.strptime(illinoisList[i], "%m-%d-%Y")-timedelta(days=1)).date().strftime("%Y-%m-%d")
    illinoisACC_dissolve["end"] = datetime.strptime(illinoisList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    
    illinoisACC_dissolve = illinoisACC_dissolve.to_crs({'init': 'epsg:4326'})
    #illinoisACC_dissolve.to_file("Illinois_ACC."+illinoisList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    illinoisACC_dissolve_list.append(illinoisACC_dissolve)
    
illinoisACC_concat = pd.concat(illinoisACC_dissolve_list, axis=0)
illinoisACC_concat = illinoisACC_concat.drop(columns=['category'])
illinoisACC_concat = illinoisACC_concat.reset_index()

#delta = timedelta(days=len(illinoisList))
firstDate = datetime.strptime(illinoisList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(illinoisList[lastIndex], "%m-%d-%Y").date()
illinoisACC_concat["dt_unit"] = "day"
illinoisACC_concat['geometry'] = illinoisACC_concat.apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x.geometry, rounding_precision=8)).simplify(0), axis = 1)

illinoisACC_concat.to_file("Illinois_ACC_v.geojson", driver="GeoJSON")

print("done")

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://

done


In [12]:
illinoisACC_concat

,category,geometry,hospital_v,start,end,dt_unit
0,0,"MULTIPOLYGON (((-88.43522 37.07011, -88.45102 ...",0.010965,2020-05-29,2020-05-30,day
1,1,"MULTIPOLYGON (((-89.17515 37.39681, -89.19079 ...",0.052221,2020-05-29,2020-05-30,day
2,2,"MULTIPOLYGON (((-90.35500 39.66639, -90.37070 ...",0.115643,2020-05-29,2020-05-30,day
3,3,"MULTIPOLYGON (((-89.07073 38.90807, -89.08670 ...",0.201564,2020-05-29,2020-05-30,day
4,4,"MULTIPOLYGON (((-88.13448 38.65059, -88.11824 ...",0.329710,2020-05-29,2020-05-30,day
...,...,...,...,...,...,...
981,1,"MULTIPOLYGON (((-88.01490 38.40386, -88.03111 ...",0.047580,2020-08-08,2020-08-09,day
982,2,"MULTIPOLYGON (((-87.96802 38.60702, -87.98429 ...",0.122471,2020-08-08,2020-08-09,day
983,3,"MULTIPOLYGON (((-88.52113 38.89400, -88.53728 ...",0.196198,2020-08-08,2020-08-09,day
984,4,"MULTIPOLYGON (((-87.98614 38.71952, -87.96984 ...",0.309341,2020-08-08,2020-08-09,day
